In [3]:
import pandas as pd
import os
import paramiko
import shutil
import datetime
import validationcheck as v

In [4]:
# Iterate through local incoming_folder, and for each CSV file, put it into the SFTP server’s upload directory

sftp_host="192.168.29.47"
sftp_username="sehaj"
sftp_password="Sehraj23"
sftp_upload_dir="/incoming_files_from_server"

local_folder=r"C:\Users\DELL\PycharmProjects\onlinekartproject\Incoming_Files\20252404"

def upload_files_to_server():
    try:
        transport=paramiko.Transport((sftp_host,22))
        transport.connect(username=sftp_username,password=sftp_password)
        sftp=paramiko.SFTPClient.from_transport(transport)
        print("SFTP connection is established")
        

        try:
           sftp.chdir(sftp_upload_dir)
           
            
        except IOError:
           print("Remote directory not found:", sftp_upload_dir)
           sftp.mkdir(sftp_upload_dir)
           sftp.chdir(sftp_upload_dir)
        print("Remote working directory:", sftp.getcwd())

        uploaded_files=False
        

        for files in os.listdir(local_folder):
            upload_filepath=os.path.join(local_folder,files)

            if os.path.isfile(upload_filepath) or files.endswith(".csv"):
                sftp.put(upload_filepath,files)
                print(f"uploaded files are {files}")
                uploaded_files=True

            else:
                print(f"skipping {files}is not a csv file")

        if not uploaded_files:
            print("no valid files exists")

        sftp.close()
        transport.close()

    except Exception as e:
        print(f" error occured {str(e)}")

upload_files_to_server()  

SFTP connection is established
Remote working directory: /C:/incoming_files_from_server
uploaded files are orders1.csv
uploaded files are orders2.csv
uploaded files are orders3.csv
uploaded files are orders4.csv
uploaded files are orders5.csv
uploaded files are orders6.csv
uploaded files are orders7.csv
uploaded files are orders8.csv


In [7]:
# Function to download files from SFTP server
def download_files_from_server(sftp_host, sftp_username, sftp_password, sftp_download_dir, local_folder):
    try:
        transport = paramiko.Transport((sftp_host, 22))
        transport.connect(username=sftp_username, password=sftp_password)
        sftp = paramiko.SFTPClient.from_transport(transport)
        print("SFTP connection is established")

        sftp.chdir(sftp_download_dir)
        files = sftp.listdir()
        

        for filename in files:
            download_path = os.path.join(local_folder, filename)
            sftp.get(filename, download_path)
            print(f"Files are downloaded: {filename}")

        sftp.close()
        transport.close()

    except Exception as e:
        print(f"Error occurred: {str(e)}")


# Main function to process the downloaded files
def main():
    try:
        today_date = datetime.date.today().strftime("%Y%d%m")
       
        # If a file exists where we want a directory, delete it
        def ensure_directory(path):
    
          if os.path.exists(path) and not os.path.isdir(path):
              print(f"Removing file blocking directory creation: {path}")
              os.remove(path)
    
          os.makedirs(path, exist_ok=True)

       
        base_dir = os.getcwd()
        incoming_file_path = os.path.join(base_dir, "incoming_files_from_server", today_date)
        success_file_path = os.path.join(base_dir, "Success_files", today_date)
        rejected_file_path = os.path.join(base_dir, "Rejected_files", today_date)

        ensure_directory(incoming_file_path)
        ensure_directory(success_file_path)
        ensure_directory(rejected_file_path)

        
        if not os.path.exists(incoming_file_path):
            print(f"Directory not found: {incoming_file_path}")
            return

        sftp_host = "192.168.29.47"
        sftp_username = "sehaj"
        sftp_password = "Sehraj23"
        sftp_download_dir = "/incoming_files_from_server"

        # Download files from the server
        download_files_from_server(sftp_host, sftp_username, sftp_password, sftp_download_dir, incoming_file_path)

        # List the downloaded files
        incoming_files = os.listdir(incoming_file_path)
        

        for file in incoming_files:
            filepath = os.path.join(incoming_file_path, file)
            print(f"The processing file is {file}")

            try:
                try:
                    df = pd.read_csv(filepath, header=0)
                    df["product_id"] = df["product_id"].astype(str)
                except pd.errors.EmptyDataError:
                    shutil.copy(filepath, rejected_file_path)
                    with open(os.path.join(rejected_file_path, f"error_{file}"), "w") as f:
                        f.write("No columns to parse — possibly malformed or headerless file\n")
                    continue

                if df.empty:
                    shutil.copy(filepath, rejected_file_path)
                    with open(os.path.join(rejected_file_path, f"error_{file}"), "w") as f:
                        f.write("File is empty\n")
                    continue

                df_products = pd.read_csv(r"master_data/product_master.csv")
                products_id = v.get_product_ids(df_products)
                product_dict = v.get_product_dict(df_products)

                df["rejected_reason"] = ""

                # Validation checks
                df.loc[~df["product_id"].apply(lambda x: v.product_id_check(x, products_id)), "rejected_reason"] += df["product_id"].apply(lambda x: f"Invalid product_id {x} |")
                df.loc[~df["city"].apply(lambda x: v.city_check(x)), "rejected_reason"] += df["city"].apply(lambda x: f"Invalid city {x} |")
                df.loc[~df["order_date"].apply(lambda x: v.order_date_check(x)), "rejected_reason"] += df["order_date"].apply(lambda x: f"Invalid order_date {x} |")

                empty_issues = df.apply(lambda row: v.empty_values_check(row.to_dict()), axis=1)
                df.loc[empty_issues.apply(len) > 0, "rejected_reason"] += empty_issues.apply(
                    lambda x: "columns " + ",".join(x) + " are empty:"
                )  
                df.loc[~df.apply(lambda row: v.sales_values_check(row.to_dict(), product_dict) if v.product_id_check(row["product_id"], products_id) else True, axis=1), "rejected_reason"] += "Wrong sales calculation |"

                
                if (df["rejected_reason"] != "").any():
    
                 shutil.copy(filepath, os.path.join(rejected_file_path, file))
                 error_path = os.path.join(rejected_file_path, f"error_{file}")
                 df.to_csv(error_path, index=False)
                 print(f"Rejected entire file due to row-level issues: {file}")
                    
                else:
    
                 shutil.copy(filepath, os.path.join(success_file_path, file))
                 print(f"Success file: {file}")

            except Exception as e:
                print(f"An error occurred while processing file {file}: {str(e)}")

        total_incoming = len(os.listdir(incoming_file_path ))
        total_success = len(os.listdir(success_file_path ))
        total_rejected = len([f for f in os.listdir(rejected_file_path )if not f.startswith("error_")])

        print(f"Total incoming files for {today_date} is {total_incoming}")
        print(f"Total suceess files for  {today_date} is {total_success}")
        print(f"Total rejected  files for {today_date} is {total_rejected}")

    
    except Exception as e:
        print(f"Main error: {str(e)}")

# Run the main function
main()



       


SFTP connection is established
Files are downloaded: orders1.csv
Files are downloaded: orders2.csv
Files are downloaded: orders3.csv
Files are downloaded: orders4.csv
Files are downloaded: orders5.csv
Files are downloaded: orders6.csv
Files are downloaded: orders7.csv
Files are downloaded: orders8.csv
The processing file is orders1.csv
Success file: orders1.csv
The processing file is orders2.csv
The processing file is orders3.csv
The processing file is orders4.csv
Rejected entire file due to row-level issues: orders4.csv
The processing file is orders5.csv
Rejected entire file due to row-level issues: orders5.csv
The processing file is orders6.csv
Success file: orders6.csv
The processing file is orders7.csv
Rejected entire file due to row-level issues: orders7.csv
The processing file is orders8.csv
Rejected entire file due to row-level issues: orders8.csv
Total incoming files for 20252404 is 8
Total suceess files for  20252404 is 2
Total rejected  files for 20252404 is 6
